In [20]:
import tifffile
import cv2
import numpy as np
import os
from pathlib import Path
import pandas as pd

def load_and_normalize_image(image_path):
    """Load and normalize image to 8-bit RGB"""
    img = tifffile.imread(image_path)
    if img.dtype != np.uint8:
        img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return img

def load_mask(mask_path, target_label):
    """Load mask and extract specific label"""
    mask = tifffile.imread(mask_path)
    # Create binary mask for specific label
    binary_mask = (mask == target_label).astype(np.uint8)
    print(len(np.unique(binary_mask)))
    return binary_mask

def create_composite_segmentation(source_img, masks_info, competitor_colors, alpha=0.3):
    """Create composite image with all segmentations"""
    overlay = np.zeros_like(source_img)
    combined_mask = np.zeros(source_img.shape[:2], dtype=np.uint8)
    print(len(masks_info))
    # Process each mask
    for mask_path, competitor, label in masks_info:
        # Load mask for specific label
        mask = load_mask(mask_path, label)
        color = competitor_colors[competitor]
        
        overlay[mask > 0] = color
        combined_mask[mask > 0] = 1
        
        # Add contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, contours, -1, color, 1)
    
    # Blend overlay with source image only once
    result = source_img.copy()
    mask_3d = np.stack([combined_mask] * 3, axis=-1)
    result = np.where(mask_3d > 0, 
                     cv2.addWeighted(overlay, alpha, source_img, 1 - alpha, 0),
                     source_img)
    
    # Re-add all contours
    for mask_path, competitor, label in masks_info:
        mask = load_mask(mask_path, label)
        color = competitor_colors[competitor]
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(result, contours, -1, color, 1)
    
    return result, combined_mask

def process_image(source_file, df, competitor_colors):
    """Process a single source file and create composite image with best segmentations"""
    
    # Load source image
    source_img = load_and_normalize_image(source_file)
    
    # Filter dataframe for this source file
    source_df = df[df['Gt_source_file'] == source_file]
    
    # Get best competitor for each label
    best_segmentations = source_df.loc[source_df.groupby('Label')['J_value'].idxmax()]
    
    # Prepare masks info
    masks_info = [(row['Mask_file'], row['competitor_name'], row['Label']) 
                 for _, row in best_segmentations.iterrows()]
    
    # Create composite image
    result = create_composite_segmentation(source_img, masks_info, competitor_colors)
    
    return result

# Create output directory
output_dir = "composite_segmentations"
os.makedirs(output_dir, exist_ok=True)

# Define colors for competitors (BGR format)
competitor_colors = {
    competitor: color for competitor, color in zip(
        df['competitor_name'].unique(),
        [
            (255, 0, 0),    # Blue
            (0, 255, 0),    # Green
            (0, 0, 255),    # Red
            (255, 255, 0),  # Cyan
            (255, 0, 255),  # Magenta
            (0, 255, 255),  # Yellow
        ]
    )
}
specific_file = "t1748.tif"

# Filter dataframe for specific file and labels
df = df[
    df['Gt_source_file'].str.contains(specific_file)]
    
# Process each unique source file
for source_file in df['Gt_source_file'].unique():
    filename = Path(source_file).name
    print(f"\nProcessing {filename}")
    
    # Create composite image
    result_image, _ = process_image(source_file, df, competitor_colors)
    # Save result
    output_path = os.path.join(output_dir, f"composite_{filename}")
    cv2.imwrite(output_path, result_image)
    
    # Create and save legend for this image
    legend_height = 50 * len(competitor_colors)
    legend_img = np.zeros((legend_height, 200, 3), dtype=np.uint8)
    legend_img.fill(255)
    
    y_position = 25
    font_scale = 0.5
    thickness = 1
    
    # Get competitors that were actually used for this image
    image_df = df[df['Gt_source_file'] == source_file]
    best_by_label = image_df.loc[image_df.groupby('Label')['J_value'].idxmax()]
    used_competitors = best_by_label['competitor_name'].unique()
    
    for competitor in used_competitors:
        color = competitor_colors[competitor]
        cv2.putText(legend_img, competitor, (60, y_position), 
                    font, font_scale, (0, 0, 0), thickness)
        cv2.rectangle(legend_img, (20, y_position-15), (50, y_position+15), 
                     color, -1)
        y_position += 50
    
    # Save legend
    legend_path = os.path.join(output_dir, f"legend_{filename}")
    cv2.imwrite(legend_path, legend_img)

print("\nProcessing complete!")
print(f"Composite images and legends saved to: {output_dir}")


Processing t1748.tif
156
1
1
2
2
2
2
2
1
2
2
1
1
1
1
2
2
2
2
2
1
2
2
2
1
1
1
1
2
2
2
1
2
2
2
2
1
2
2
2
2
2
2
2
2
1
2
2
1
1
2
2
2
2
2
2
2
2
2
1
2
2
2
2
1
2
2
2
2
2
2
2
2
1
1
2
2
1
2
2
2
2
2
2
2
2
2
1
2
2
2
2
1
2
1
2
2
2
2
2
2
2
2
2
1
2
1
2
1
2
2
2
1
2
1
2
2
2
1
2
2
2
2
2
1
1
1
2
2
2
2
1
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
2
2
2
2
1
1
1
2
2
2
2
2
1
2
2
1
1
1
1
2
2
2
2
2
1
2
2
2
1
1
1
1
2
2
2
1
2
2
2
2
1
2
2
2
2
2
2
2
2
1
2
2
1
1
2
2
2
2
2
2
2
2
2
1
2
2
2
2
1
2
2
2
2
2
2
2
2
1
1
2
2
1
2
2
2
2
2
2
2
2
2
1
2
2
2
2
1
2
1
2
2
2
2
2
2
2
2
2
1
2
1
2
1
2
2
2
1
2
1
2
2
2
1
2
2
2
2
2
1
1
1
2
2
2
2
1
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
2
2
2
2
1

Processing complete!
Composite images and legends saved to: composite_segmentations


In [2]:
df = pd.read_csv('dataset.csv')

In [ ]:
specific_df

In [10]:
import tifffile
import cv2
import numpy as np
import os
from pathlib import Path
import pandas as pd

def load_and_normalize_image(image_path):
    """Load and normalize image to 8-bit RGB"""
    img = tifffile.imread(image_path)
    if img.dtype != np.uint8:
        img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return img

def load_mask(mask_path, target_label):
    """Load mask and extract specific label"""
    mask = tifffile.imread(mask_path)
    # Create binary mask for specific label
    binary_mask = (mask == target_label).astype(np.uint8)
    return binary_mask

def create_composite_segmentation(source_img, masks_info, competitor_colors, alpha=0.3):
    """Create composite image with all segmentations"""
    overlay = np.zeros_like(source_img)
    combined_mask = np.zeros(source_img.shape[:2], dtype=np.uint8)
    
    # Process each mask
    for mask_path, competitor, label in masks_info:
        # Load mask for specific label
        mask = load_mask(mask_path, label)
        color = competitor_colors[competitor]
        
        overlay[mask > 0] = color
        combined_mask[mask > 0] = 1
        
        # Add contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, contours, -1, color, 1)
    
    # Blend overlay with source image only once
    result = source_img.copy()
    mask_3d = np.stack([combined_mask] * 3, axis=-1)
    result = np.where(mask_3d > 0, 
                     cv2.addWeighted(overlay, alpha, source_img, 1 - alpha, 0),
                     source_img)
    
    # Re-add all contours
    for mask_path, competitor, label in masks_info:
        mask = load_mask(mask_path, label)
        color = competitor_colors[competitor]
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(result, contours, -1, color, 1)
    
    return result, combined_mask

def get_bounding_box(mask, margin=50):
    """Get bounding box coordinates with margin"""
    y_indices, x_indices = np.where(mask > 0)
    if len(y_indices) == 0 or len(x_indices) == 0:
        return None
    
    min_x = max(0, np.min(x_indices) - margin)
    max_x = min(mask.shape[1], np.max(x_indices) + margin)
    min_y = max(0, np.min(y_indices) - margin)
    max_y = min(mask.shape[0], np.max(y_indices) + margin)
    
    return (min_x, min_y, max_x, max_y)

# Create special output directory
special_output_dir = "visualizations/special_segmentations_t1748"
os.makedirs(special_output_dir, exist_ok=True)

# Specific labels we want to show
specific_labels = [1, 5, 11, 29, 48, 69, 103, 113, 141, 148]
specific_file = "t1748.tif"

# Filter dataframe for specific file and labels
specific_df = df[
    (df['Gt_source_file'].str.contains(specific_file)) & 
    (df['Label'].isin(specific_labels))
]

if len(specific_df) > 0:
    # Load source image
    source_file = specific_df['Gt_source_file'].iloc[0]
    source_img = load_and_normalize_image(source_file)
    
    # Get best competitor for each label
    best_segmentations = specific_df.loc[specific_df.groupby('Label')['J_value'].idxmax()]
    
    # Prepare masks info
    masks_info = [(row['Mask_file'], row['competitor_name'], row['Label']) 
                 for _, row in best_segmentations.iterrows()]
    
    # Create composite image
    result, combined_mask = create_composite_segmentation(source_img, masks_info, competitor_colors)
    
    # Save full-size result
    full_output_path = os.path.join(special_output_dir, f"full_size_composite_{specific_file}")
    cv2.imwrite(full_output_path, result)
    
    # Get bounding box for zoomed version
    bbox = get_bounding_box(combined_mask, margin=50)
    if bbox is not None:
        min_x, min_y, max_x, max_y = bbox
        zoomed_result = result[min_y:max_y, min_x:max_x]
        
        # Save zoomed result
        zoomed_output_path = os.path.join(special_output_dir, f"zoomed_composite_{specific_file}")
        cv2.imwrite(zoomed_output_path, zoomed_result)
    
    # Create and save legend
    legend_height = 50 * len(best_segmentations)
    legend_img = np.zeros((legend_height, 300, 3), dtype=np.uint8)
    legend_img.fill(255)
    
    y_position = 25
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    thickness = 1
    
    for _, row in best_segmentations.iterrows():
        competitor = row['competitor_name']
        label = row['Label']
        color = competitor_colors[competitor]
        text = f"Label {label} - {competitor}"
        
        cv2.putText(legend_img, text, (60, y_position), 
                    font, font_scale, (0, 0, 0), thickness)
        cv2.rectangle(legend_img, (20, y_position-15), (50, y_position+15), 
                     color, -1)
        y_position += 50
    
    # Save legend
    legend_path = os.path.join(special_output_dir, f"legend_{specific_file}")
    cv2.imwrite(legend_path, legend_img)
    
    print(f"\nProcessing complete! Files saved to: {special_output_dir}")
    print(f"Created:")
    print(f"- Full size composite")
    print(f"- Zoomed composite")
    print(f"- Legend with label information")
else:
    print(f"No matching data found for file {specific_file} with specified labels")


Processing complete! Files saved to: visualization/special_segmentations_t1748
Created:
- Full size composite
- Zoomed composite
- Legend with label information
